# REQUIRED LIBRARIES INSTALLATION

In [1]:
pip install cmake

   ---------------------------------------- 0.0/36.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/36.7 MB ? eta -:--:--
    --------------------------------------- 0.8/36.7 MB 4.2 MB/s eta 0:00:09
   -- ------------------------------------- 2.1/36.7 MB 4.7 MB/s eta 0:00:08
   --- ------------------------------------ 3.1/36.7 MB 5.0 MB/s eta 0:00:07
   ---- ----------------------------------- 3.9/36.7 MB 4.9 MB/s eta 0:00:07
   ----- ---------------------------------- 5.2/36.7 MB 4.8 MB/s eta 0:00:07
   ------ --------------------------------- 6.0/36.7 MB 4.6 MB/s eta 0:00:07
   ------- -------------------------------- 7.1/36.7 MB 4.8 MB/s eta 0:00:07
   -------- ------------------------------- 7.9/36.7 MB 4.8 MB/s eta 0:00:06
   ---------- ----------------------------- 9.2/36.7 MB 4.8 MB/s eta 0:00:06
   ----------- ---------------------------- 10.2/36.7 MB 4.8 MB/s eta 0:00:06
   ------------ --------------------------- 11.5/36.7 MB 4.9 MB/s eta 0:00:06
   --------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install dlib

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install face-recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0.tar.gz (100.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached dlib-19.24.8.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Created wheel for dlib: filename=dlib-19.24.8-cp312-cp312-win_amd64.whl size=3177380 sha256=734c5ebc3a73570a8965585d9eda8df4e4df6a4e74271fafa400e75725f7bd6b
  Stored in directory: c:\users\a06bh\appdata\local\pip\cache\wheels\45\47\c4\f5c2cf6f3b4ab4f558f27375101ead0ef7f1861d5aa54f5f8c
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566178 sha256=57df69b32b04c779e554dc7c6045b4b76d86064a5c3bb6b4570135f32265a8c2
  Stored in directory: c:\users\a06bh\appdata\local\pip\cache\wheels


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install cvzone

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26309 sha256=e725b286495da473577ea5b49977419894aeed3739cbf1b9155cd0c41c74b706
  Stored in directory: c:\users\a06bh\appdata\local\pip\cache\wheels\5d\21\e8\3147ae88d44e27f06e0175d337a7673c70fb957202cbbe2034
Successfully built cvzone
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install opencv-python


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## CONNECTING WEBCAM

In [1]:
import cv2
import os
import pickle
import face_recognition
import numpy as np
import cvzone

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

# Load background image
original_bg = cv2.imread('Resources/bg.png')
imgbg = original_bg.copy()

# Load mode images
folderModePath = 'Resources/Modes'
modePathList = os.listdir(folderModePath)
imgModeList = [cv2.imread(os.path.join(folderModePath, path)) for path in modePathList]

# Display parameters
webcam_width = 480
webcam_height = 360
mode_img_width = 350
mode_img_index = 0

# Load encodings
print("Loading Encodings")
with open('EncodeGenerator.p', 'rb') as file:
    encodeListKnownWithIds = pickle.load(file)
encodeListKnown, studentIds = encodeListKnownWithIds
print("Encode file loaded")

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture webcam frame.")
        break

    # Reset background
    imgbg = original_bg.copy()

    # Face detection
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

    # Resize and position webcam image
    resized_img = cv2.resize(img, (webcam_width, webcam_height))
    bg_height, bg_width = imgbg.shape[:2]
    x_off = (bg_width - webcam_width) // 2
    y_off = (bg_height - webcam_height) // 2
    imgbg[y_off:y_off + webcam_height, x_off:x_off + webcam_width] = resized_img

    # Position mode image
    # mode_img_resized = cv2.resize(imgModeList[mode_img_index], (mode_img_width, webcam_height))
    # imgbg[y_off:y_off + webcam_height, x_off + webcam_width:x_off + webcam_width + mode_img_width] = mode_img_resized

    # Face recognition
    for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        # Default label
        label = "Unknown"
        box_color = (0, 0, 255)  # Red for unknown
        
        if len(faceDis) > 0:
            matchIndex = np.argmin(faceDis)
            if matches[matchIndex]:
                label = str(studentIds[matchIndex])
                box_color = (0, 255, 0)  # Green for known
            
            # Scale and position bounding box
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            bbox_x = x_off + x1 - 80
            bbox_y = y_off + y1 - 80
            bbox_w = x2 - x1 
            bbox_h = y2 - y1 
            
            # Draw bounding box
            imgbg = cvzone.cornerRect(imgbg, (bbox_x, bbox_y, bbox_w, bbox_h), rt=0, colorR=box_color)
            
            # Add label with background
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(imgbg, 
                         (bbox_x, bbox_y - text_height - 10), 
                         (bbox_x + text_width, bbox_y - 10), 
                         (0, 0, 0), -1)
            cv2.putText(imgbg, label, 
                       (bbox_x, bbox_y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("Face Recognition", imgbg)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loading Encodings
Encode file loaded
